In [1]:
!pip install gTTS SpeechRecognition python-weather beautifulsoup4 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1


In [8]:
import os
import time
import requests
import json
import python_weather
import asyncio
from gtts import gTTS
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode
from datetime import datetime
from bs4 import BeautifulSoup

# --- JS Code for Microphone Access in Colab ---
RECORD_JS = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  // Specify mimeType for WAV format
  recorder = new MediaRecorder(stream, { mimeType: 'audio/wav' })
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks, { type: 'audio/wav' })
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def speak(text):
    """Converts text to speech and plays it in Colab."""
    print(f"Assistant: {text}")
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    display(Audio("response.mp3", autoplay=True))

async def get_weather(city="London"):
    async with python_weather.Client(unit=python_weather.IMPERIAL) as client:
        weather = await client.get(city)
        return f"The current temperature in {city} is {weather.temperature} degrees."

def get_news():
    url = 'https://news.google.com/rss'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'xml')
    titles = [item.title.text for item in soup.find_all('item')[:3]]
    return "Here are the top three headlines: " + ". ".join(titles)

# Global list for simple reminders
reminders = []

def process_command(text):
    text = text.lower()
    if 'weather' in text:
        # Simple extraction: assumes "weather in [City]"
        city = text.split("in")[-1].strip() if "in" in text else "New York"
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(get_weather(city))

    elif 'news' in text:
        return get_news()

    elif 'reminder' in text or 'remind' in text:
        reminders.append(text.replace("remind me to", "").strip())
        return "Okay, I've added that to your reminders."

    elif 'time' in text:
        return f"The current time is {datetime.now().strftime('%H:%M')}"

    else:
        return "I heard you say: " + text + ". I can help with weather, news, or reminders."

def run_assistant():
    display(Javascript(RECORD_JS))
    speak("I am listening. Please speak now.")

    # Record for 4 seconds
    s = output.eval_js('record(4000)')
    b = b64decode(s.split(',')[1])

    with open('audio.wav','wb') as f:
        f.write(b)

    import speech_recognition as sr
    r = sr.Recognizer()
    with sr.AudioFile('audio.wav') as source:
        audio = r.record(source)

    try:
        user_text = r.recognize_google(audio)
        print(f"You: {user_text}")
        response = process_command(user_text)
        speak(response)
    except Exception as e:
        speak("Sorry, I couldn't understand the audio.")

# Run the assistant
run_assistant()

<IPython.core.display.Javascript object>

Assistant: I am listening. Please speak now.


KeyboardInterrupt: 